<a href="https://colab.research.google.com/github/rornellas/tech-challenge-3-ia4devs/blob/main/fine_tunning_tech_challenge_fase_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install -q transformers datasets
!pip install -q triton

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━

# Executando no COLAB utilizando GPU

In [1]:
from unsloth import FastLanguageModel
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

#torch.cuda.is_available()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


# Processando arquivo de input

In [2]:
import json
from itertools import islice

# Carregar os dados
with open('/content/drive/MyDrive/IA para devs - fiap/Fase 3/Tech Challenge/trn_first10000.json', 'r', encoding='utf-8') as f:
    rows = [
        {
            'title': entry.get('title', ''),
            'description': entry.get('content', '')
        }
        for entry in (json.loads(line) for line in islice(f, 200))
        if len(entry.get('content', '')) > len(entry.get('title', ''))
    ]

#print("Tamanho rows1: ", rows1.__len__())

#rows = [item for item in rows1 for _ in range(2)]

print("Tamanho rows: ", rows.__len__())

# Exibir a primeira entrada filtrada, se houver
if rows:
    print(json.dumps(rows[0], indent=4, ensure_ascii=False))
else:
    print("Nenhum registro corresponde ao critério.")


Tamanho rows:  95
{
    "title": "Girls Ballet Tutu Neon Pink",
    "description": "High quality 3 layer ballet tutu. 12 inches in length"
}


## Realizando limpeza

A limpeza acaba não sendo necessária pois estamos utilizando o tokenizer para montar o treino e executar os prompts

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import json
import html

# Baixar recursos do NLTK
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('brown')


def normalize_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
    return text

def remove_punctuation(text):
    if isinstance(text, str):
        text = re.sub(r'[^\w\s]', '', text)
    return text

stop_words = set(stopwords.words('english'))  # Ajuste para 'portuguese' se necessário

def remove_stopwords(text):
    if isinstance(text, str):
        tokens = nltk.word_tokenize(text)
        tokens = [word for word in tokens if word not in stop_words]
        text = ' '.join(tokens)
    return text

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    if isinstance(text, str):
        tokens = nltk.word_tokenize(text)
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        text = ' '.join(tokens)
    return text

#def correct_spelling(text):
#    if isinstance(text, str):
#        blob = TextBlob(text)
#        corrected = str(blob.correct())
#        return corrected
#    return text

def preprocessarDado(text):
  text = html.unescape(text)
  text = normalize_text(text)
  #text = remove_punctuation(text)
  #text = remove_stopwords(text)
  #text = lemmatize_text(text)
  return text



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [4]:
import pandas as pd

df = pd.DataFrame(rows)

# Remover duplicatas
df = df.drop_duplicates(subset=['title', 'description'])

df['title'] = df['title'].apply(preprocessarDado)
df['description'] = df['description'].apply(preprocessarDado)

# Remover entradas vazias
df = df.dropna(subset=['title', 'description'])
df = df[(df['title'].str.strip() != '') & (df['description'].str.strip() != '')]

# Converter para array JSON atribuído a uma variável
data = df.to_dict(orient='records')

data[0]


{'title': 'girls ballet tutu neon pink',
 'description': 'high quality 3 layer ballet tutu 12 inches length'}

## Criando Modelo para Treinamento

In [5]:
from unsloth import FastLanguageModel

# Nome do modelo a ser utilizado
model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [6]:
import gc
import torch

try:
    del model
except:
    pass

try:
    del tokenizer
except:
    pass

gc.collect()
torch.cuda.empty_cache()

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post4: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [7]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Provide a response that provides more information about the input context. Keep the response to a single paragraph.

### Input:
{}

### Response:
{}
"""

In [17]:
#special_tokens = {'additional_special_tokens': ['<s>', '</s>', '[INST]', '[/INST]', '<<SYS>>', '<</SYS>>']}
#tokenizer.add_special_tokens(special_tokens)

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, max_length=2048)
pass

def formatting_prompts_func(examples):
    inputs       = examples["title"]
    outputs      = examples["description"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = tokenizer.bos_token + prompt.format(input, output) + tokenizer.eos_token
        texts.append(text)
    return {"text": texts}
pass

from datasets import Dataset

dataset = Dataset.from_list(data)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/95 [00:00<?, ? examples/s]

In [18]:
idx_example = 65

# Dividindo o dataset em 80% treino e 20% validação
#train_test_split = dataset.train_test_split(test_size=0.2, seed=42)

# Separando em datasets de treino e validação
#train_dataset = train_test_split['train']
#eval_dataset = train_test_split['test']
train_dataset = dataset
print(train_dataset[idx_example])
#print("\n")
#print(eval_dataset[0])

{'title': 'blood royal story spencers royals', 'description': 'lively superficial overview lives times princess dianas illustrious family 500 years tudor age present gaining losing vast fortunes making shrewd statusenhancing marriages jockeying social political power recurring themes family saga whose subjects rose wealthy farmers reign henry vii ranking among prosperous prominent families land 18th century see wealth status decline throughout victorian contemporary erasuntil diana third daughter eighth earl made fairytale marriage royal family including notables sarah churchill infamous 18thcentury duchess marlborough whose carefully calculated scheming made family one richest land spencer clan replete colorful characters princess diana drew strength ancestral heritage pearson claims undoubtable dubious however repeated assertions various facets dianas character direct inheritances one ancestor another suggestion biology destiny culminates propositionbased observation princes william 

In [19]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [20]:
def runGPT(modelToRun, input, temperature=0.5, top_p=0.9):
  FastLanguageModel.for_inference(modelToRun)
  inputs = tokenizer(
  [
      prompt.format(
          input,
          "",
      )
  ], return_tensors = "pt").to("cuda")

  from transformers import TextStreamer
  text_streamer = TextStreamer(tokenizer)
  _ = modelToRun.generate(**inputs, streamer = text_streamer, max_new_tokens = 256, top_p = top_p, temperature = temperature)

## Executando antes do treino

In [21]:
runGPT(model, train_dataset[idx_example]['title'])

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Provide a response that provides more information about the input context. Keep the response to a single paragraph.

### Input:
blood royal story spencers royals

### Response:

The "Blood Royal Story: Spencers, Royals" likely refers to a narrative that delves into the history and connection of the Spencer family, who are one of the most prominent British aristocratic families, and the British Royal Family. The Spencers have a significant link to the royals, as Charles, Prince of Wales, married Diana, Princess of Wales, who was a member of the Spencer family. This union brought the two families even closer, and the Spencer name has been associated with the British Royal Family ever since. The story would likely explore the history, traditions, and relationships between these two influential families.</s>


### Descrição contida no dataset

In [22]:
train_dataset[idx_example]['description']

'lively superficial overview lives times princess dianas illustrious family 500 years tudor age present gaining losing vast fortunes making shrewd statusenhancing marriages jockeying social political power recurring themes family saga whose subjects rose wealthy farmers reign henry vii ranking among prosperous prominent families land 18th century see wealth status decline throughout victorian contemporary erasuntil diana third daughter eighth earl made fairytale marriage royal family including notables sarah churchill infamous 18thcentury duchess marlborough whose carefully calculated scheming made family one richest land spencer clan replete colorful characters princess diana drew strength ancestral heritage pearson claims undoubtable dubious however repeated assertions various facets dianas character direct inheritances one ancestor another suggestion biology destiny culminates propositionbased observation princes william harry look like mother fatherthat spencer genes superseded win

In [23]:
from transformers import TrainerCallback, TrainerControl, TrainerState

class EarlyStoppingByLoss(TrainerCallback):
    """
    Callback para parar o treinamento quando o training loss atinge um determinado limiar.
    """
    def __init__(self, threshold: float):
        """
        Args:
            threshold (float): O valor da perda de treinamento para parar o treinamento.
        """
        self.threshold = threshold

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None:
            return

        # Obtém a perda de treinamento
        training_loss = logs.get("loss")

        if training_loss is not None:
            #print(f"Perda de treinamento atual: {training_loss}")
            if training_loss <= self.threshold:
                print(f"Perda de treinamento atingiu o limite de {self.threshold}. Parando o treinamento.")
                control.should_training_stop = True

In [27]:
base_location = "/content/drive/MyDrive/IA para devs - fiap/Fase 3/Tech Challenge/20240930/"

In [28]:
# a tentativa de treinar com um validation não estava dando um resultado muito legal...

def criarSFTTrainer(datasetParam, epochs = 10, learningRate=3e-5, trainBatchsize=4, accumulationSteps=8, threshold=1, ):
  return SFTTrainer(
      model=model,
      tokenizer=tokenizer,
      train_dataset=datasetParam,  # Certifique-se de que o dataset foi carregado corretamente
      #eval_dataset=eval_dataset,  # Certifique-se de que o dataset foi carregado corretamente
      dataset_text_field="text",
      max_seq_length=max_seq_length,
      dataset_num_proc=2,
      packing=False,
      callbacks=[EarlyStoppingByLoss(threshold=threshold)],
      args=TrainingArguments(
          output_dir=base_location+"checkpoints",        # Diretório para salvar checkpoints
          save_steps=50,                    # Salvar a cada 500 steps
          save_total_limit=5,
          per_device_train_batch_size=trainBatchsize,  # Ajuste o batch size conforme a memória disponível
          gradient_accumulation_steps=accumulationSteps,  # Gradiente acumulado para simular um batch maior
          num_train_epochs = epochs,
          #warmup_steps=15,  # Passos de aquecimento da taxa de aprendizado
          #max_steps=maxSteps,  # Aumente o número de steps se necessário
          learning_rate=learningRate,  # Taxa de aprendizado
          fp16 = not torch.cuda.is_bf16_supported(),
          bf16 = torch.cuda.is_bf16_supported(),# Usar BF16 se disponível (para A100)
          logging_steps=2,  # Intervalo para logging
          optim="adamw_8bit",  # AdamW em 8 bits para otimizar memória
          #optim="lion_8bit",  # implementação nova baseada no adamw, com melhor performance e convergencia em determinados cenários
          #aparentemente o lion consome menos memória, tem conversao mais estavel e treina mais rapido
          weight_decay=0.1,  # Regularização
          lr_scheduler_type="linear",  # Scheduler linear para a taxa de aprendizado
          seed=3407,  # Semente para reprodutibilidade
          #evaluation_strategy="steps",  # Avaliar a cada n steps
          #eval_steps=10,  # Avaliar a cada 5 steps
      ),
  )

In [29]:
try:
    del trainer_stats
except:
    pass

try:
    del trainer
except:
    pass

gc.collect()

134

In [30]:
trainer = criarSFTTrainer(train_dataset, 20, 2e-4, 4, 8, 1)

Map (num_proc=2):   0%|          | 0/95 [00:00<?, ? examples/s]

In [31]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 95 | Num Epochs = 20
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 8
\        /    Total batch size = 32 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
2,4.407600
4,3.210300
6,2.815800
8,2.529200
10,2.338000
12,2.106000
14,1.883200
16,1.812600
18,1.635100
20,1.356900


Perda de treinamento atingiu o limite de 1. Parando o treinamento.


In [36]:
import random

idx = random.randint(0, len(train_dataset)-1)

print("idx = ", idx)

print("title: "+train_dataset[idx]['title'])
print("description: "+train_dataset[idx]['description'])

idx =  16
title: day life china
description: ninety photographers 60 western 30 chinese fanned across china single dayapril 15 1989to take pictures make rare candid glimpse worlds populous nation photos show us villages farms take us homes schools factories kaleidoscope imagesriceplanting brigades defendant dock trial murder yaks main street one citywe get feel contradictions pressures modernizing society many photos exceptionally beautiful taken together reveal diversity vast country moving tentlike yurts mongolian nomads tantric buddhist monastery occupied tibet photoessay concludes glimpse protest posters went beijing university students began occupy tiananmen square tragically ironic conclusion light events followed first serial time author tourcopyright 1989 reed business information inc


In [37]:
runGPT(model, train_dataset[idx]['title'])

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Provide a response that provides more information about the input context. Keep the response to a single paragraph.

### Input:
day life china

### Response:

day life china

a vivid portrait of contemporary china, this book offers a unique and intimate look at the daily lives of its people. from the bustling streets of beijing to the remote villages of yunnan, the author captures the essence of china in all its complexity and diversity. through a series of evocative photographs and insightful essays, the reader is transported to a world far removed from their own, yet one that is surprisingly familiar.

the book begins with a journey through the ancient city of xi'an, where the author encounters the famous terracotta warriors and the bustling markets of the old city. from there, the journey continues to the 

In [ ]:
runGPT(model, "what do you know about \"el war\" mercenaries?")

In [ ]:
lora_model_dir = "lora_model_mistral-instruct"

In [ ]:
model.save_pretrained(base_location+"trained_model/"+lora_model_dir) # Local saving
tokenizer.save_pretrained(base_location+"trained_model/"+lora_model_dir)


In [ ]:
def loadModel(model_dir):
  if True:
      from unsloth import FastLanguageModel
      model, tokenizer = FastLanguageModel.from_pretrained(
          model_name = model_dir,
          max_seq_length = max_seq_length,
          dtype = dtype,
          load_in_4bit = load_in_4bit,
      )
      FastLanguageModel.for_inference(model)



In [ ]:
reloadedModel = loadModel(base_location+"trained_model/"+lora_model_dir)

runGPT(reloadedModel, train_dataset[0]['title'])

## Retreinando um modelo já processado anteriormente

In [ ]:
trainer_stats = treinar(model, 50, 4, 8).train()